In [2]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Grid parameters
grid_h, grid_w = 50, 50
radius = 1
n_points = 8 * radius

def extract_features_from_cell(cell):
    features = []
    # Intensity
    features.append(np.mean(cell))
    features.append(np.std(cell))
    
    # LBP
    lbp = local_binary_pattern(cell, n_points, radius, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points+3), density=True)
    features.extend(hist)
    
    # Edge density
    edges = cv2.Canny(cell, 50, 150)
    features.append(np.sum(edges > 0) / cell.size)
    
    return np.array(features)

def extract_features_from_image(img):
    features_list = []
    h, w = img.shape
    for y in range(0, h, grid_h):
        for x in range(0, w, grid_w):
            cell = img[y:y+grid_h, x:x+grid_w]
            if cell.shape[0] != grid_h or cell.shape[1] != grid_w:
                cell = cv2.resize(cell, (grid_w, grid_h))
            features_list.append(extract_features_from_cell(cell))
    return np.vstack(features_list)

def load_images(image_folder):
    images = []
    for fname in os.listdir(image_folder):
        path = os.path.join(image_folder, fname)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img = cv2.resize(img, (800, 600))
        images.append(img)
    return images

# Main pipeline
if __name__ == "__main__":
    image_folder = "images"
    images = load_images(image_folder)

    # Extract features for all grids
    all_features = []
    grid_indices = []
    for idx, img in enumerate(images):
        feats = extract_features_from_image(img)
        all_features.append(feats)
        grid_indices.append((idx, feats.shape[0]))
    all_features = np.vstack(all_features)

    # Scale features
    scaler = StandardScaler()
    all_features_scaled = scaler.fit_transform(all_features)

    # Unsupervised detection
    iso = IsolationForest(contamination=0.1, random_state=42)
    iso.fit(all_features_scaled)
    preds = iso.predict(all_features_scaled)  # -1 = anomaly (wildlife), 1 = normal

    # Map predictions back to images
    start = 0
    for img_idx, n_grids in grid_indices:
        grid_pred = preds[start:start+n_grids]
        start += n_grids
        num_rows = 600 // grid_h
        num_cols = 800 // grid_w
        grid_map = (grid_pred == -1).astype(int).reshape((num_rows, num_cols))
        print(f"Image {img_idx} grid map (1 = wildlife):")
        print(grid_map)


Image 0 grid map (1 = wildlife):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Image 1 grid map (1 = wildlife):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Image 2 grid map (1 = wildlife):
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 